In [32]:
source_layer = "ra"
source_data = "Seven_Day_Ahead_Demand_Forecast"
source_container = "sink"
source_account = "dapffuksrawdl01"
source_file_path = "demand_forecasting/7_Day_Ahead_Demand_Forecast/CSV/NG_demand_7DA/2023/11/05"
#Printing the path
path = 'abfss://%s@%s.dfs.core.windows.net/%s' % (source_container, source_account, source_file_path)
print(path)

StatementMeta(aspool, 272335, 43, Finished, Available)

abfss://sink@dapffuksrawdl01.dfs.core.windows.net/demand_forecasting/7_Day_Ahead_Demand_Forecast/CSV/NG_demand_7DA/2023/11/05


In [33]:
#Loading the file into a dataframe
Seven_Day_Ahead_Demand_Forecast_raw = spark.read.load('abfss://%s@%s.dfs.core.windows.net/%s' % (source_container, source_account, source_file_path), format='csv',Header= 'True')
display(Seven_Day_Ahead_Demand_Forecast_raw)
print(Seven_Day_Ahead_Demand_Forecast_raw.count())

StatementMeta(aspool, 272335, 44, Finished, Available)

SynapseWidget(Synapse.DataFrame, 388125e1-4f43-4914-81f8-54193eb58998)

11


In [34]:
from pyspark.sql.functions import concat_ws

Seven_Day_Ahead_Demand_Forecast_raw=Seven_Day_Ahead_Demand_Forecast_raw.withColumn("CUSTOM_ID", concat_ws("_","TARGETDATE","CARDINALPOINT"))
display(Seven_Day_Ahead_Demand_Forecast_raw)
print(Seven_Day_Ahead_Demand_Forecast_raw.count())

StatementMeta(aspool, 272335, 45, Finished, Available)

SynapseWidget(Synapse.DataFrame, b394b114-3ad3-43f6-8398-137039a1a8f3)

11


In [35]:
Seven_Day_Ahead_Demand_Forecast_raw.printSchema()

StatementMeta(aspool, 272335, 46, Finished, Available)

root
 |-- DAYSAHEAD: string (nullable = true)
 |-- TARGETDATE: string (nullable = true)
 |-- FORECASTDEMAND: string (nullable = true)
 |-- CARDINALPOINT: string (nullable = true)
 |-- CP_TYPE: string (nullable = true)
 |-- CP_ST_TIME: string (nullable = true)
 |-- CP_END_TIME: string (nullable = true)
 |-- F_Point: string (nullable = true)
 |-- CUSTOM_ID: string (nullable = false)



In [36]:
from datetime import datetime
import pyspark.sql.types as T
import pyspark.sql.functions as F
def user_defined_timestamp(date_col):
   _date = datetime.strptime(date_col, '%Y%m%d')
   return _date.strftime('%Y-%m-%d')
user_defined_timestamp_udf = F.udf(user_defined_timestamp, T.StringType())
Seven_Day_Ahead_Demand_Forecast_raw = Seven_Day_Ahead_Demand_Forecast_raw.withColumn('TARGETDATE_DATE', user_defined_timestamp_udf('TARGETDATE'))
#from pyspark.sql.functions import concat_ws
display(Seven_Day_Ahead_Demand_Forecast_raw)


StatementMeta(aspool, 272335, 47, Finished, Available)

SynapseWidget(Synapse.DataFrame, b3b5fc46-1baa-4429-8309-ae5ae5ea6208)

In [37]:
#Dropping some of the columns as they are externally added in Cleansed layer Seven_Day_Ahead_Demand_Forecast_cleansed
cl_cols_to_drop_raw = ["TARGETDATE"]
#sc_lsoa_cleansed_reconc=sc_lsoa_cleansed.sort("CUSTOM_ID")
Seven_Day_Ahead_Demand_Forecast_raw_new = Seven_Day_Ahead_Demand_Forecast_raw.drop(*cl_cols_to_drop_raw)
display(Seven_Day_Ahead_Demand_Forecast_raw_new)

StatementMeta(aspool, 272335, 48, Finished, Available)

SynapseWidget(Synapse.DataFrame, e3115029-604b-4a06-900d-c57c67c3d2a2)

In [38]:
from pyspark.sql.types import  TimestampType, TimestampType,IntegerType,IntegerType
Seven_Day_Ahead_Demand_Forecast_raw=Seven_Day_Ahead_Demand_Forecast_raw_new.withColumnRenamed("TARGETDATE_DATE","TARGETDATE")
print(Seven_Day_Ahead_Demand_Forecast_raw.count())


StatementMeta(aspool, 272335, 49, Finished, Available)

11


In [39]:
# changing the schema of columns from string to respective data types as mentioned in cleansed layer
from pyspark.sql.types import DateType,IntegerType,TimestampType
Seven_Day_Ahead_Demand_Forecast_raw_new = Seven_Day_Ahead_Demand_Forecast_raw \
            .withColumn("DAYSAHEAD" ,
              Seven_Day_Ahead_Demand_Forecast_raw["DAYSAHEAD"]
              .cast(IntegerType()))  \
              .withColumn("TARGETDATE" ,
              Seven_Day_Ahead_Demand_Forecast_raw["TARGETDATE"]
              .cast(DateType()))  \
            .withColumn("FORECASTDEMAND" ,
              Seven_Day_Ahead_Demand_Forecast_raw["FORECASTDEMAND"]
              .cast(IntegerType()))  \
            .withColumn("CP_ST_TIME" ,
              Seven_Day_Ahead_Demand_Forecast_raw["CP_ST_TIME"]
              .cast(IntegerType()))  \
            .withColumn("CP_END_TIME" ,
              Seven_Day_Ahead_Demand_Forecast_raw["CP_END_TIME"]
              .cast(IntegerType()))  
Seven_Day_Ahead_Demand_Forecast_raw.printSchema() 
Seven_Day_Ahead_Demand_Forecast_raw_new.printSchema()                             
display(Seven_Day_Ahead_Demand_Forecast_raw_new)
print(Seven_Day_Ahead_Demand_Forecast_raw_new.count())

StatementMeta(aspool, 272335, 50, Finished, Available)

root
 |-- DAYSAHEAD: string (nullable = true)
 |-- FORECASTDEMAND: string (nullable = true)
 |-- CARDINALPOINT: string (nullable = true)
 |-- CP_TYPE: string (nullable = true)
 |-- CP_ST_TIME: string (nullable = true)
 |-- CP_END_TIME: string (nullable = true)
 |-- F_Point: string (nullable = true)
 |-- CUSTOM_ID: string (nullable = false)
 |-- TARGETDATE: string (nullable = true)

root
 |-- DAYSAHEAD: integer (nullable = true)
 |-- FORECASTDEMAND: integer (nullable = true)
 |-- CARDINALPOINT: string (nullable = true)
 |-- CP_TYPE: string (nullable = true)
 |-- CP_ST_TIME: integer (nullable = true)
 |-- CP_END_TIME: integer (nullable = true)
 |-- F_Point: string (nullable = true)
 |-- CUSTOM_ID: string (nullable = false)
 |-- TARGETDATE: date (nullable = true)



SynapseWidget(Synapse.DataFrame, 9ac9b614-204b-43bb-8a32-165ef864242b)

11


In [40]:
target_layer = "cl"
target_source = "Seven_Day_Ahead_Demand_Forecast"
target_container = "datatransformation"
target_account = "dapffukscleanseddl01"
target_file_path = "energy_balance/7day_ahead_demand_forecast/ng_demand_7da/v1_deltalake"

StatementMeta(aspool, 272335, 51, Finished, Available)

In [41]:
Seven_Day_Ahead_Demand_Forecast_cleansed = spark.read.load('abfss://%s@%s.dfs.core.windows.net/%s/' % (target_container, target_account, target_file_path), format='delta')
display(Seven_Day_Ahead_Demand_Forecast_cleansed)
print(Seven_Day_Ahead_Demand_Forecast_cleansed.count())

StatementMeta(aspool, 272335, 52, Finished, Available)

SynapseWidget(Synapse.DataFrame, ded243cb-280a-47e9-ba76-e1d554e22b25)

572


In [42]:
print(Seven_Day_Ahead_Demand_Forecast_raw_new.count())
print(Seven_Day_Ahead_Demand_Forecast_cleansed.count())

StatementMeta(aspool, 272335, 53, Finished, Available)

11
572


In [43]:
%run /Tools/Utilities

StatementMeta(, , -1, Finished, Available)

..........Running Utilities Notebook.........


In [44]:
display(Seven_Day_Ahead_Demand_Forecast_cleansed.groupBy('_SRCFILENAME').count().where(col('count')>1))

StatementMeta(aspool, 272335, 65, Finished, Available)

SynapseWidget(Synapse.DataFrame, a4c6c4aa-d02b-4cd3-82ef-5e42427c27b9)

In [45]:
Seven_Day_Ahead_Demand_Forecast_cleansed_re=Seven_Day_Ahead_Demand_Forecast_cleansed.filter(Seven_Day_Ahead_Demand_Forecast_cleansed._SRCFILENAME=='/sink/demand_forecasting/7_Day_Ahead_Demand_Forecast/CSV/NG_demand_7DA/2023/11/05/NG_demand_7DA_20231112_202311051615_202311051615.csv?version=1699201110518?flength=540')
print(Seven_Day_Ahead_Demand_Forecast_cleansed_re.count())
display(Seven_Day_Ahead_Demand_Forecast_cleansed_re)

StatementMeta(aspool, 272335, 66, Finished, Available)

11


SynapseWidget(Synapse.DataFrame, 8d873078-7264-4db4-a8e4-bcf4fd5d1a6f)

In [46]:
Seven_Day_Ahead_Demand_Forecast_raw_new.printSchema()
Seven_Day_Ahead_Demand_Forecast_cleansed_re.printSchema()

StatementMeta(aspool, 272335, 67, Finished, Available)

root
 |-- DAYSAHEAD: integer (nullable = true)
 |-- FORECASTDEMAND: integer (nullable = true)
 |-- CARDINALPOINT: string (nullable = true)
 |-- CP_TYPE: string (nullable = true)
 |-- CP_ST_TIME: integer (nullable = true)
 |-- CP_END_TIME: integer (nullable = true)
 |-- F_Point: string (nullable = true)
 |-- CUSTOM_ID: string (nullable = false)
 |-- TARGETDATE: date (nullable = true)

root
 |-- CUSTOM_ID: string (nullable = false)
 |-- DAYSAHEAD: integer (nullable = true)
 |-- TARGETDATE: date (nullable = false)
 |-- FORECASTDEMAND: integer (nullable = true)
 |-- CARDINALPOINT: string (nullable = false)
 |-- CP_TYPE: string (nullable = true)
 |-- CP_ST_TIME: integer (nullable = true)
 |-- CP_END_TIME: integer (nullable = true)
 |-- F_POINT: string (nullable = true)
 |-- _BATCHID: string (nullable = true)
 |-- _SRCFILENAME: string (nullable = true)
 |-- _CREATEDON: timestamp (nullable = true)
 |-- _UPDATEDON: timestamp (nullable = true)



In [47]:
#Sorting the order of columns of RTL in RAW Layer
Seven_Day_Ahead_Demand_Forecast_raw_new = Seven_Day_Ahead_Demand_Forecast_raw_new[['CUSTOM_ID','DAYSAHEAD','TARGETDATE','FORECASTDEMAND','CARDINALPOINT','CP_TYPE','CP_ST_TIME','CP_END_TIME','F_POINT',]]
#display(Seven_Day_Ahead_Demand_Forecast_raw_new)

StatementMeta(aspool, 272335, 68, Finished, Available)

In [48]:
#Seven_Day_Ahead_Demand_Forecast_raw_new.printSchema()
#Seven_Day_Ahead_Demand_Forecast_cleansed.printSchema()

StatementMeta(aspool, 272335, 69, Finished, Available)

In [49]:
#display(Seven_Day_Ahead_Demand_Forecast_raw_new)
#display(Seven_Day_Ahead_Demand_Forecast_cleansed_re)

StatementMeta(aspool, 272335, 70, Finished, Available)

In [50]:
#Dropping some of the columns as they are externally added in Cleansed layer Seven_Day_Ahead_Demand_Forecast_cleansed
cl_cols_to_drop_cleansed = ["_BATCHID","_SRCFILENAME","_CREATEDON","_UPDATEDON"]
#sc_lsoa_cleansed_reconc=sc_lsoa_cleansed.sort("CUSTOM_ID")
Seven_Day_Ahead_Demand_Forecast_cleansed_new = Seven_Day_Ahead_Demand_Forecast_cleansed_re.drop(*cl_cols_to_drop_cleansed)
display(Seven_Day_Ahead_Demand_Forecast_cleansed_new)

StatementMeta(aspool, 272335, 71, Finished, Available)

SynapseWidget(Synapse.DataFrame, 9559a25c-bdd1-4ccb-860e-f2cf0ee89642)

In [51]:
#print(Seven_Day_Ahead_Demand_Forecast_raw_new.count())
#print(Seven_Day_Ahead_Demand_Forecast_cleansed_new.count())
#Seven_Day_Ahead_Demand_Forecast_raw_new.printSchema()
#Seven_Day_Ahead_Demand_Forecast_cleansed_new.printSchema()

StatementMeta(aspool, 272335, 72, Finished, Available)

In [52]:
#Null check for CUSTOM_ID
CUSTOM_ID_null_check = Seven_Day_Ahead_Demand_Forecast_cleansed.filter(Seven_Day_Ahead_Demand_Forecast_cleansed.CUSTOM_ID.isNull()).count()
print(CUSTOM_ID_null_check)

assert CUSTOM_ID_null_check == 0
#Null check for _BATCHID
_BATCHID_null_check = Seven_Day_Ahead_Demand_Forecast_cleansed.filter(Seven_Day_Ahead_Demand_Forecast_cleansed._BATCHID.isNull()).count()
print(_BATCHID_null_check)

assert _BATCHID_null_check == 0
#Null check for _SRCFILENAME
_SRCFILENAME_null_check = Seven_Day_Ahead_Demand_Forecast_cleansed.filter(Seven_Day_Ahead_Demand_Forecast_cleansed._SRCFILENAME.isNull()).count()
print(_SRCFILENAME_null_check)

assert _SRCFILENAME_null_check == 0
_CREATEDON_null_check =Seven_Day_Ahead_Demand_Forecast_cleansed.filter(Seven_Day_Ahead_Demand_Forecast_cleansed._CREATEDON.isNull()).count()
print(_CREATEDON_null_check)

assert _CREATEDON_null_check == 0
#Null check for _UPDATEDON
_UPDATEDON_null_check =Seven_Day_Ahead_Demand_Forecast_cleansed.filter(Seven_Day_Ahead_Demand_Forecast_cleansed._UPDATEDON.isNull()).count()
print(_UPDATEDON_null_check)

assert _UPDATEDON_null_check == 0
#Null check for TARGETDATE
TARGETDATE_null_check =Seven_Day_Ahead_Demand_Forecast_cleansed.filter(Seven_Day_Ahead_Demand_Forecast_cleansed.TARGETDATE.isNull()).count()
print(TARGETDATE_null_check)

assert TARGETDATE_null_check == 0
#Null check for CARDINALPOINT
CARDINALPOINT_null_check =Seven_Day_Ahead_Demand_Forecast_cleansed.filter(Seven_Day_Ahead_Demand_Forecast_cleansed.CARDINALPOINT.isNull()).count()
print(CARDINALPOINT_null_check)

assert CARDINALPOINT_null_check == 0


StatementMeta(aspool, 272335, 73, Finished, Available)

0
0
0
0
0
0
0


In [53]:
print(Seven_Day_Ahead_Demand_Forecast_cleansed_new.count())
print(Seven_Day_Ahead_Demand_Forecast_cleansed_new.dropDuplicates(["CUSTOM_ID"]).count())

StatementMeta(aspool, 272335, 74, Finished, Available)

11
11


In [54]:
Seven_Day_Ahead_Demand_Forecast_raw_new1=Seven_Day_Ahead_Demand_Forecast_raw_new.sort('CUSTOM_ID')
display(Seven_Day_Ahead_Demand_Forecast_raw_new1)
Seven_Day_Ahead_Demand_Forecast_cleansed_new1=Seven_Day_Ahead_Demand_Forecast_cleansed_new.sort('CUSTOM_ID')
display(Seven_Day_Ahead_Demand_Forecast_cleansed_new1)

StatementMeta(aspool, 272335, 75, Finished, Available)

SynapseWidget(Synapse.DataFrame, 72424917-4423-4e8c-9004-65b294442a8f)

SynapseWidget(Synapse.DataFrame, cef5bac6-4986-4f1e-9547-531bc4075792)

In [55]:
Seven_Day_Ahead_Demand_Forecast_raw_new1.printSchema()
Seven_Day_Ahead_Demand_Forecast_cleansed_new1.printSchema()

StatementMeta(aspool, 272335, 76, Finished, Available)

root
 |-- CUSTOM_ID: string (nullable = false)
 |-- DAYSAHEAD: integer (nullable = true)
 |-- TARGETDATE: date (nullable = true)
 |-- FORECASTDEMAND: integer (nullable = true)
 |-- CARDINALPOINT: string (nullable = true)
 |-- CP_TYPE: string (nullable = true)
 |-- CP_ST_TIME: integer (nullable = true)
 |-- CP_END_TIME: integer (nullable = true)
 |-- F_POINT: string (nullable = true)

root
 |-- CUSTOM_ID: string (nullable = false)
 |-- DAYSAHEAD: integer (nullable = true)
 |-- TARGETDATE: date (nullable = false)
 |-- FORECASTDEMAND: integer (nullable = true)
 |-- CARDINALPOINT: string (nullable = false)
 |-- CP_TYPE: string (nullable = true)
 |-- CP_ST_TIME: integer (nullable = true)
 |-- CP_END_TIME: integer (nullable = true)
 |-- F_POINT: string (nullable = true)



In [56]:
data_reconc = Seven_Day_Ahead_Demand_Forecast_raw_new1.exceptAll(Seven_Day_Ahead_Demand_Forecast_cleansed_new1).unionAll(Seven_Day_Ahead_Demand_Forecast_cleansed_new1.exceptAll(Seven_Day_Ahead_Demand_Forecast_raw_new1))
print(data_reconc.count())
display(data_reconc)

StatementMeta(aspool, 272335, 77, Finished, Available)

22


SynapseWidget(Synapse.DataFrame, be400599-d533-4b24-aa4e-f3e34f7c4cf9)

In [57]:
print(Seven_Day_Ahead_Demand_Forecast_raw_new.count())
print(Seven_Day_Ahead_Demand_Forecast_cleansed_new.count())
display(Seven_Day_Ahead_Demand_Forecast_raw_new1)
display(Seven_Day_Ahead_Demand_Forecast_cleansed_new)

Seven_Day_Ahead_Demand_Forecast_raw_new1_reconc=Seven_Day_Ahead_Demand_Forecast_raw_new1
Seven_Day_Ahead_Demand_Forecast_cleansed_new_reconc=Seven_Day_Ahead_Demand_Forecast_cleansed_new

StatementMeta(aspool, 272335, 78, Finished, Available)

11
11


SynapseWidget(Synapse.DataFrame, b5042d43-4e14-4c7d-9ce7-5f2d418c6022)

SynapseWidget(Synapse.DataFrame, d5f4f7f7-eda9-4d41-9abc-cc2df1322afc)

In [58]:
Seven_Day_Ahead_Demand_Forecast_cleansed_new_reconc.createOrReplaceTempView('Cleansed')

StatementMeta(aspool, 272335, 79, Finished, Available)

In [59]:
Seven_Day_Ahead_Demand_Forecast_raw_new1_reconc.createOrReplaceTempView('raw')

StatementMeta(aspool, 272335, 80, Finished, Available)

In [61]:
%%sql
select *,'Cl' from Cleansed where CUSTOM_ID='20231112_1B'
Union All
select *,'R' from raw where CUSTOM_ID='20231112_1B'

StatementMeta(aspool, 272335, 82, Finished, Available)

<Spark SQL result set with 2 rows and 10 fields>